In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio

In [2]:
#full runtime indicator start
full_time=time.time()

In [3]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

In [4]:
#notebook complete indicator
notebook = '../../Data/Hugo Good Job.mp3'
complete=Audio(url = notebook, autoplay = True)

In [5]:
map = gpd.read_file('../../Data/Taxi Zones Shape/taxi_zones.shp').to_crs('epsg:4326')
map = map.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)


complete

In [6]:
df=pd.read_csv('../../Data/Taxi Data/yellow_14/yellow_tripdata_2014-11.csv')

C:\Users\Bro Bro\.conda\envs\geospatial\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


df.info()

df2=df.head(100)

df2.columns

In [7]:
start_time = time.time()

df['dropoff_geometry'] = df.apply(lambda x: Point((float(x[' dropoff_longitude']), 
                                             float(x[' dropoff_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 811.2050583362579 seconds ---


In [8]:
start_time = time.time()

df['pickup_geometry'] = df.apply(lambda x: Point((float(x[' pickup_longitude']), 
                                             float(x[' pickup_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 807.5019402503967 seconds ---


In [9]:
df.head(3)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,dropoff_geometry,pickup_geometry
0,VTS,2014-11-01 00:47:00,2014-11-01 01:00:00,5,2.71,-73.960960,40.818780,1,NaN,-73.923620,40.808400,CRD,11.5,0.5,0.5,2.40,0.0,14.90,POINT (-73.92362 40.8084),POINT (-73.96096 40.81878)
1,VTS,2014-11-01 00:28:00,2014-11-01 01:00:00,5,4.67,-73.990148,40.729315,1,NaN,-73.954933,40.769335,CRD,23.5,0.5,0.5,6.00,0.0,30.50,POINT (-73.95493299999998 40.769335),POINT (-73.990148 40.729315)
2,VTS,2014-11-01 00:39:00,2014-11-01 01:00:00,2,4.52,-73.980122,40.742847,1,NaN,-73.977007,40.784257,CRD,17.0,0.5,0.5,4.38,0.0,22.38,POINT (-73.977007 40.784257),POINT (-73.98012199999998 40.742847)


In [10]:
df2_start=df[[' passenger_count',' trip_distance',' fare_amount',' mta_tax',' tip_amount',' tolls_amount','pickup_geometry']]

In [11]:
df2_end=df[['dropoff_geometry']]

In [12]:
start_time = time.time()
df2_start=gpd.GeoDataFrame(df2_start,
                     crs=map.crs,
                     geometry=df2_start['pickup_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 106.86182284355164 seconds ---


In [13]:
start_time = time.time()
df2_end=gpd.GeoDataFrame(df2_end,
                     crs=map.crs,
                     geometry=df2_end['dropoff_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 106.98476386070251 seconds ---


In [14]:
start_time = time.time()

geo_processing_start = gpd.sjoin(df2_start, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1064.4458286762238 seconds ---


In [15]:
start_time = time.time()

geo_processing_end = gpd.sjoin(df2_end, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1067.7544889450073 seconds ---


In [16]:
geo_processing_start

,passenger_count,trip_distance,fare_amount,mta_tax,tip_amount,tolls_amount,pickup_geometry,geometry,index_right,zone,LocationID,borough
0,5,2.71,11.5,0.5,2.40,0.00,POINT (-73.96096 40.81878),POINT (-73.96096 40.81878),151,Manhattanville,152,Manhattan
353,1,3.10,14.5,0.5,0.00,0.00,POINT (-73.95410099999998 40.821604),POINT (-73.95410 40.82160),151,Manhattanville,152,Manhattan
1536,1,7.50,27.0,0.5,5.50,0.00,POINT (-73.95434199999998 40.821317),POINT (-73.95434 40.82132),151,Manhattanville,152,Manhattan
2462,1,18.10,52.0,0.5,5.00,5.33,POINT (-73.954261 40.816034),POINT (-73.95426 40.81603),151,Manhattanville,152,Manhattan
2544,2,5.90,28.0,0.5,8.55,0.00,POINT (-73.95432099999998 40.821409),POINT (-73.95432 40.82141),151,Manhattanville,152,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...
9755358,1,1.84,7.5,0.5,1.60,0.00,POINT (-74.140895 40.632907),POINT (-74.14090 40.63291),186,Port Richmond,187,Staten Island
6669122,1,0.00,8.0,0.0,0.00,0.00,POINT (-73.899665 40.838192),POINT (-73.89966 40.83819),58,Crotona Park,59,Bronx
11267783,1,8.30,24.5,0.5,3.50,5.33,POINT (-73.895583 40.837013),POINT (-73.89558 40.83701),58,Crotona Park,59,Bronx
10183868,1,5.50,18.5,0.5,3.90,0.00,POINT (-74.139792 40.548129),POINT (-74.13979 40.54813),108,Great Kills,109,Staten Island


In [17]:
group_start = geo_processing_start.groupby(['LocationID']).describe()

In [18]:
group_start.columns = ['_'.join(col) for col in group_start.columns.values]

group_start.to_csv('../../Data/yellow_tripdata_2014-2-start-processed.csv')

In [19]:
group_end = geo_processing_end.value_counts(['LocationID'])

In [20]:
group_end = pd.DataFrame(group_end,columns=['dropoff_count'])

group_end.to_csv('../../Data/yellow_tripdata_2014-2-end-processed.csv')

In [21]:
merge=group_start.merge(group_end, how='left', on='LocationID')

In [22]:
#REMEMBER TO RENAME CSV BEFORE START
merge.to_csv('../../Data/yellow_tripdata_2014-11-processed.csv')
complete

In [23]:
print("--- %s seconds ---" % (time.time() - full_time))

--- 4009.822827577591 seconds ---
